In [1]:
# Necessary imports
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np
import spacy

In [30]:
# Dataset from Peggy's previous MedCab4 build

url1 = 'https://raw.githubusercontent.com/PeggyK1/med_cab3/main/data/strains.csv'
df1 = pd.read_csv(url1)
# Dropping unneeded column
df1 = df1.drop(columns = ['Unnamed: 0'])
#Preping it for tokenizatiopn
df1['name'] = df1['name'].replace('-', ' ', regex=True)
df1['name'] = df1['name'].str.upper()

df1.head()

,id,name,type,effects,ailment,flavor,location,thc,cbd,Citrus,Sweet,Earthy,Pine,Skunk,Berry,Lemon,Lime,Blueberry,Strawberry,Mango,Rose,Pepper,Depression,Pain,Insomnia,Stress,Lack of Appetite,Muscle Spasms,Nausea,Inflammation,Relaxed,Happy,Euphoric,Uplifted,Sleepy,Dry Mouth,Focused,Energetic,Paranoid,Anxious,Hungry,Talkative,Creative
0,1,100 OG,Hybrid,Focused,Depression,Citrus,Unknown,127,16,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,5,AFGHANI BULLRIDER,Hybrid,"Uplifted, Relaxed, Happy, Euphoric, Dry Mouth,...","Stress, Depression, Insomnia, Pain","Sweet, Pine, Earthy",Unknown,127,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,0,1,0,0,1,0,0,0,0
2,20,ALOHA,Hybrid,"Energetic, Uplifted, Happy, Creative, Focused,...","Depression, Stress, Pain","Sweet, Citrus",Unknown,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,0,1,1,1,0,1,0,0,1
3,24,AMNESIA HAZE,Hybrid,"Happy, Uplifted, Euphoric, Energetic, Creative...","Stress, Depression, Pain","Citrus, Lemon, Earthy",Unknown,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,1,1,0,1,0,1,1,1,0,0,1
4,35,BC SWEET TOOTH,Hybrid,"Uplifted, Happy, Relaxed, Sleepy, Euphoric, Dr...","Stress, Insomnia, Depression, Nausea, Pain","Sweet, Honey",Unknown,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,1,1,1,1,1,1,0,0,1,0,0,0,0


In [31]:
# Model I found from a previous study guide
url2 = 'https://raw.githubusercontent.com/bundickm/Study-Guides/master/data/cannabis.csv'
df2 = pd.read_csv(url2)

# Prepping for tokenization
df2['Strain'] = df2['Strain'].replace('-', ' ', regex=True)
df2['Strain'] = df2['Strain'].str.upper()
df2 =df2.rename(columns = {'Strain':'name'})

df2.head()

,name,Type,Rating,Effects,Flavor,Description
0,100 OG,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98 WHITE WIDOW,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13 DAWGS,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K GOLD,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [32]:
# Merging the two datasets
strains = pd.merge(df1, df2, on='name')
# Dropping NaN values
strains = strains.dropna()
# Dropping duplicate and unneeded columns
strains = strains.drop(columns = ['location', 'Type', 'Effects', 'Flavor'])

strains.head()

,id,name,type,effects,ailment,flavor,thc,cbd,Citrus,Sweet,Earthy,Pine,Skunk,Berry,Lemon,Lime,Blueberry,Strawberry,Mango,Rose,Pepper,Depression,Pain,Insomnia,Stress,Lack of Appetite,Muscle Spasms,Nausea,Inflammation,Relaxed,Happy,Euphoric,Uplifted,Sleepy,Dry Mouth,Focused,Energetic,Paranoid,Anxious,Hungry,Talkative,Creative,Rating,Description
0,1,100 OG,Hybrid,Focused,Depression,Citrus,127,16,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0,$100 OG is a 50/50 hybrid strain that packs a ...
1,20,ALOHA,Hybrid,"Energetic, Uplifted, Happy, Creative, Focused,...","Depression, Stress, Pain","Sweet, Citrus",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,0,1,1,1,0,1,0,0,1,4.2,Aloha is a sativa strain that leaves users wit...
2,24,AMNESIA HAZE,Hybrid,"Happy, Uplifted, Euphoric, Energetic, Creative...","Stress, Depression, Pain","Citrus, Lemon, Earthy",0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,1,1,0,1,0,1,1,1,0,0,1,4.3,"With earthy flavors of lemons and citrus, Amne..."
3,35,BC SWEET TOOTH,Hybrid,"Uplifted, Happy, Relaxed, Sleepy, Euphoric, Dr...","Stress, Insomnia, Depression, Nausea, Pain","Sweet, Honey",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,1,1,1,1,1,1,0,0,1,0,0,0,0,4.3,"Developed in British Columbia by BC Bud Depot,..."
4,36,BERKELEY,Hybrid,"Talkative, Happy, Uplifted, Energetic, Focused...","Stress, Depression, Lack of Appetite, Pain","Citrus, Sweet",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,1,0,1,0,1,1,1,1,1,0,1,0,4.3,Berkeley is rumored to be the super-potent bot...


In [5]:
# Tokenizing function
def tokenize(text):
    """Parses a string into a list of semantic units (words)

    Args:
        text (str): The string that the function will tokenize.

    Returns:
        list: tokens parsed out by the mechanics of your choice
    """
    
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = tokens.lower().split()
    
    return tokens

# Tokenized columns
strains['name_tokens'] = (strains['name']).apply(tokenize)
strains['ailment_tokens'] = strains['ailment'].apply(tokenize)
strains['type_tokens'] = strains['type'].apply(tokenize)
strains['effects_tokens'] = strains['effects'].apply(tokenize)
strains['flavor_tokens'] = strains['flavor'].apply(tokenize)

In [33]:
# create the transformer
vect = CountVectorizer(stop_words='english', 
                       max_df=.95,
                       max_features=1500,
                      ngram_range=(1,3),
                      tokenizer=tokenize)

data = strains['ailment'] + strains['type'] + strains['effects'] + strains['flavor']

# transform text
dtm1 = vect.fit_transform(data)
dtm1 = pd.DataFrame(dtm1.todense(), columns=vect.get_feature_names())

dtm1.head()

,ammonia,anxious,anxious dry,anxious dry mouth,anxious dry mouthsweet,anxious paranoidberry,anxious paranoidberry sweet,anxious paranoidcitrus,anxious paranoidcitrus earthy,anxious paranoidcitrus sweet,anxious paranoidearthy,anxious paranoidearthy citrus,anxious paranoidearthy pine,anxious paranoidearthy sweet,anxious paranoidlemon,anxious paranoidpine,anxious paranoidsweet,anxious paranoidsweet earthy,anxiousberry,anxiousberry sweet,anxiousberry sweet blueberry,anxiousberry sweet earthy,anxiousblueberry,anxiousblueberry sweet,anxiouscitrus,anxiouscitrus earthy,anxiouscitrus orange,anxiouscitrus orange sweet,anxiouscitrus sweet,anxiousearthy,anxiousearthy berry,anxiousearthy citrus,anxiousearthy coffee,anxiousearthy grape,anxiousearthy pine,anxiousearthy skunk,anxiousearthy sweet,anxiousearthy sweet pine,anxiousgrape,anxiousgrape earthy,...,uplifted euphoric hungry,uplifted euphoric relaxed,uplifted euphoric sleepy,uplifted euphoric talkative,uplifted euphoric tingly,uplifted focused,uplifted focused creative,uplifted focused dry,uplifted focused energetic,uplifted focused euphoric,uplifted focused happy,uplifted focused sleepy,uplifted happy,uplifted happy creative,uplifted happy dry,uplifted happy energetic,uplifted happy euphoric,uplifted happy focused,uplifted happy hungry,uplifted happy sleepy,uplifted happy talkative,uplifted horny,uplifted horny dry,uplifted hungry,uplifted hungry dry,uplifted paranoid,uplifted relaxed,uplifted relaxed creative,uplifted relaxed dry,uplifted relaxed energetic,uplifted relaxed euphoric,uplifted relaxed talkative,uplifted sleepy,uplifted sleepy dry,uplifted talkative,uplifted talkative dry,uplifted talkative energetic,uplifted tingly,uplifted tingly dry,vanilla
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# Instantiating vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=9000)

# Creating vocab and getting word counts per document
dtm2 = tfidf.fit_transform(data)
dtm2 = pd.DataFrame(dtm2.todense(), columns=tfidf.get_feature_names())

dtm2.head()

,ammonia,anxious,anxiousammonia,anxiousberry,anxiousblueberry,anxiouscitrus,anxiouscoffee,anxiousearthy,anxiousgrape,anxioushoney,anxiouslemon,anxiouslime,anxiousmango,anxiouspine,anxiouspineapple,anxiousrose,anxiousskunk,anxiousstrawberry,anxioussweet,anxiousvanilla,appetite,appetitehybridenergetic,appetitehybrideuphoric,appetitehybridhappy,appetitehybridrelaxed,appetitehybridsleepy,appetitehybridtalkative,appetitehybriduplifted,appetiteindicaeuphoric,appetiteindicahappy,appetiteindicahorny,appetiteindicahungry,appetiteindicarelaxed,appetiteindicasleepy,appetiteindicauplifted,appetitesativacreative,appetitesativaenergetic,appetitesativaeuphoric,appetitesativafocused,appetitesativahappy,...,stresshybriduplifted,stressindicaeuphoric,stressindicafocused,stressindicahappy,stressindicahorny,stressindicahungry,stressindicarelaxed,stressindicasleepy,stressindicatalkative,stressindicatingly,stressindicauplifted,stresssativacreative,stresssativaenergetic,stresssativaeuphoric,stresssativafocused,stresssativahappy,stresssativahorny,stresssativarelaxed,stresssativasleepy,stresssativatalkative,stresssativatingly,stresssativauplifted,sweet,talkative,talkativeberry,talkativecitrus,talkativepine,talkativesweet,tingly,tinglycitrus,tinglyearthy,tinglylemon,tinglymint,tinglysweet,uplifted,upliftedammonia,upliftedearthy,upliftedpepper,upliftedsweet,vanilla
0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.422092,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.188147,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.43907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166472,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.307018,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.257353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.216046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.156637,0.0,0.0,0.0,0.0,0.0


In [34]:
# A test ailment
my_ailment = ["My back has been hurting for three months, and it makes it hard to go to sleep."]

# Transforming the ailment
new = vect.transform(my_ailment)

# Building the model and fitting it
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm1)

# Testing the test ailment
nn.kneighbors(new.todense())
names = strains['name']
names[147]

'DURGA MATA'

### Cutoff: Above is Brad's Work from Monday

In [36]:
feature = strains['ailment'] + strains['type'] + strains['effects'] + strains['flavor']
target = strains['name']

In [23]:
features = tfidf.fit_transform(feature)
features = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())

In [24]:
targets = tfidf.fit_transform(target)
targets = pd.DataFrame(targets.todense(), columns=tfidf.get_feature_names())

In [25]:
print(features.shape)
print(targets.shape)

(882, 341)
(882, 796)


In [26]:
p1 = nn.fit(features, targets)

In [ ]:
# Pickling model
import pickle

filename = 'first_model.sav'
pickle.dump(p1, open(filename, 'wb'))